<a href="https://colab.research.google.com/github/mitiau/nlp_epitopes/blob/main/epitope_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependecies and download weights

In [2]:
!pip install transformers
!pip install fair-esm
!pip install biopython
!pip install gdown

In [3]:
!gdown --id 1VaOtlK389p4XDfPfiEb8Xt1EMdGghJDI
!gdown --id 1doP6d6K6O5DHzi0EhXYqeNVHChVDB9cu
!gdown --id 1KYS8y28I1Ul4sXInWufwxi560-h2nahD
!gdown --id 1Z5wEwnZgn3oUGubBVUbOy6fOuiA2F5Dh
!gdown --id 1P0zm_AcRFCAE3Y8w_25yoyyME4q1eqCQ
!gdown --id 15-WU5YuslcYPS76C8LVUGBBBHSURTCkR

Downloading...
From: https://drive.google.com/uc?id=1VaOtlK389p4XDfPfiEb8Xt1EMdGghJDI
To: /content/esm1v_ft_epitopes_0.pth
100% 2.61G/2.61G [00:29<00:00, 87.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1doP6d6K6O5DHzi0EhXYqeNVHChVDB9cu
To: /content/esm1v_ft_epitopes_1.pth
100% 2.61G/2.61G [00:52<00:00, 49.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KYS8y28I1Ul4sXInWufwxi560-h2nahD
To: /content/esm1v_ft_epitopes_2.pth
100% 2.61G/2.61G [00:52<00:00, 50.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Z5wEwnZgn3oUGubBVUbOy6fOuiA2F5Dh
To: /content/esm1v_ft_epitopes_3.pth
100% 2.61G/2.61G [00:54<00:00, 48.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1P0zm_AcRFCAE3Y8w_25yoyyME4q1eqCQ
To: /content/esm1v_ft_epitopes_4.pth
100% 2.61G/2.61G [00:54<00:00, 47.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=15-WU5YuslcYPS76C8LVUGBBBHSURTCkR
To: /content/dummy.fasta
100% 26.8k/26.8k [00:00<00:00, 22.0MB/s]


In [1]:
from google.colab import drive, files

import torch
from torch.utils.data import Dataset
from torch import nn

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput

import pandas as pd
import numpy as np

import esm
from esm import ProteinBertModel
from esm.pretrained import load_model_and_alphabet_hub

from Bio import SeqIO
from io import StringIO, BytesIO
from tqdm import tqdm

In [2]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels
        
        #args = Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='[0.9,0.98]', adam_eps=1e-08, all_gather_list_size=16384, arch='roberta_large', attention_dropout=0.0, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_suffix='', checkpoint_transformer_block=False, clip_norm=0.0, cpu=False, criterion='masked_lm', curriculum=0, data='/checkpoint/bioseq_nonsecure/data/clustering/scaling_lm_data_clustering/ur_clust__ur_tp__max_seqs_300/out', data_buffer_size=10, dataset_impl='fasta', ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method='tcp://learnfair1333:19522', distributed_no_spawn=False, distributed_port=19522, distributed_rank=0, distributed_world_size=128, distributed_wrapper='DDP', dropout=0.0, effective_attention=False, empty_cache_freq=0, encoder_attention_heads=20, encoder_embed_dim=1280, encoder_ffn_embed_dim=5120, encoder_layerdrop=0, encoder_layers=33, encoder_layers_to_keep=None, encoder_learned_pos=True, encoder_normalize_after=True, encoder_normalize_before=False, fast_stat_sync=False, find_unused_parameters=False, finetune_from_model=None, fix_batches_to_gpus=False, fixed_validation_seed=None, fp16=True, fp16_init_scale=4, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, freq_weighted_replacement=False, heartbeat_timeout=3600, keep_best_checkpoints=-1, keep_interval_updates=-1, keep_last_epochs=-1, layer_norm_fp32=False, leave_unmasked_prob=0.1, localsgd_frequency=3, log_format='json', log_interval=100, lr=[0.0004], lr_scheduler='inverse_sqrt', mask_prob=0.15, mask_whole_words=False, max_epoch=200, max_positions=1024, max_sentences=None, max_sentences_valid=None, max_tokens=1024, max_tokens_valid=1024, max_update=0, maximize_best_checkpoint_metric=False, memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, min_lr=1e-09, model_parallel_size=1, no_epoch_checkpoints=False, no_last_checkpoints=False, no_mid_epoch_validate=False, no_progress_bar=False, no_save=False, no_save_optimizer_state=False, no_seed_provided=False, nprocs_per_node=8, num_batch_buckets=0, num_workers=3, optimizer='adam', optimizer_overrides='{}', patience=-1, pooler_activation_fn='tanh', pooler_dropout=0.0, preact_normalize=True, profile=False, quant_noise_pq=0, quant_noise_pq_block_size=8, quant_noise_scalar=0, quantization_config_path=None, random_token_prob=0.1, required_batch_size_multiple=8, reset_dataloader=False, reset_lr_scheduler=False, reset_meters=False, reset_optimizer=False, restore_file='checkpoint_last.pt', sample_break_mode='eos', save_dir='/checkpoint/bioseq_nonsecure/esm1b/MLM/exp1/700M.uniref_2020.redo_ur90.train90.adam[0.9_0.98].seed2', save_interval=1000000, save_interval_updates=5000, scoring='bleu', seed=2, sentence_avg=False, share_decoder_input_output_embed=False, shorten_data_split_list='train', shorten_method='random_crop', skip_invalid_size_inputs_valid_test=True, slowmo_algorithm='LocalSGD', slowmo_momentum=None, stop_time_hours=0, task='masked_lm', tensorboard_logdir='/checkpoint/bioseq_nonsecure/esm1b/MLM/exp1/tb/700M.uniref_2020.redo_ur90.train90.adam[0.9_0.98].seed2', threshold_loss_scale=None, token_dropout=True, tokenizer=None, tokens_per_sample=1023, tpu=False, train_subset='train90', untie_weights_robert=False, untie_weights_roberta=False, update_freq=[8], use_bert_init=True, use_bmuf=False, use_inf_norm=False, use_old_adam=False, user_dir=None, valid_subset='valid30,valid50,valid90', validate_after_updates=0, validate_interval=1000000, validate_interval_updates=0, warmup_init_lr=1e-07, warmup_updates=16000, weight_decay=0.0, zero_sharding='none')
        
        self.esm1v, self.esm1v_alphabet = load_model_and_alphabet_hub("esm1v_t33_650M_UR90S_1")
        #self.esm1v = ProteinBertModel(args, esm.Alphabet.from_architecture('roberta_large'))


        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)
    
class pred_Dataset(Dataset):
    def __init__(self, df):
        self.df = df
        esm1v_alphabet = esm.Alphabet.from_architecture('roberta_large')
        #_, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
        self.esm1v_batch_converter = esm1v_alphabet.get_batch_converter()

    def __getitem__(self, idx):
        _, _, esm1b_batch_tokens = self.esm1v_batch_converter([('' , ''.join(self.df.iloc[idx,1])[:1022])])
        return {'token_ids': esm1b_batch_tokens}
    
    def __len__(self):
        return len(self.df)

In [3]:
model = ESM1vForTokenClassification()

/usr/local/lib/python3.7/dist-packages/esm/pretrained.py:134: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn("Regression weights not found, predicting contacts will not produce correct results.")


# Compute epitopes

In [4]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving seqs_A222_Y144_correct.fasta to seqs_A222_Y144_correct (2).fasta
User uploaded file "seqs_A222_Y144_correct.fasta" with length 52443 bytes


In [5]:


for key in uploaded.keys():
  identifiers = []
  seqs = []
  for seq_record in SeqIO.parse(StringIO(BytesIO(uploaded[key]).read().decode('UTF-8')), 'fasta'):  # (generator)
    identifiers.append(seq_record.id)
    seqs.append(seq_record.seq)

  pred_df = pd.DataFrame({'id':identifiers, 'seq': seqs})
  pred_ds = pred_Dataset(pred_df)

  res=[]
  for ens_idx in range(5):
    
    model.load_state_dict(torch.load('esm1v_ft_epitopes_'+str(ens_idx) + '.pth'))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(pred_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)

  merged=[]
  for it_idx in range(len(res[0])):
    merged.append(np.mean(np.stack([res[i][it_idx] for i in range(5)], axis =0), axis =0))
    
  pred_df['epitope_prediction'] = merged
  pred_df[['id', 'epitope_prediction']].to_pickle(key + '.preds.pkl')

  0%|          | 0/41 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
from google.colab import files
for key in uploaded.keys():
  files.download(key + '.preds.pkl') 